In [68]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [69]:
# Load and normalizde the data

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batchSize = 5
validSize = 0.2 # use 20% of train set as validation

trainValidSet = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testSet = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainSet, validSet = torch.utils.data.random_split(trainValidSet, [int(len(trainValidSet)*(1-validSize)), int(len(trainValidSet)*validSize)])

trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=batchSize, shuffle=True)
validLoader = torch.utils.data.DataLoader(validSet, batch_size=batchSize, shuffle=True)
testLoader = torch.utils.data.DataLoader(testSet, batch_size=batchSize, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [70]:
# Writer for tensorBoard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [71]:
len(trainLoader)*batchSize, len(validLoader)*batchSize, len(testLoader)*batchSize, next(iter(testLoader))[0][0].shape

(40000, 10000, 10000, torch.Size([3, 32, 32]))

In [72]:
# Define the network class

class ConvNet(nn.Module):
    def __init__(self, activation):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 100)
        self.fc2 = nn.Linear(100, 10)
        self.activation = activation

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x


In [73]:
def trainNetwork(network, optimizer, lossFunction, epochs, writer):
    network.train()
    for epoch in range(epochs):

        ### TRAINING ###
        trainLoss = 0
        for batch_nr, (images, labels) in enumerate(trainLoader):

            # Predict
            predictions = network(images)

            # Get loss and backpropogate
            loss = lossFunction(predictions, labels)
            loss.backward() 

            # Optimize parameters (weights and biases) and remove gradients after
            optimizer.step() 
            optimizer.zero_grad()

            # Save loss for whole epoch
            trainLoss += loss.item()

        trainLoss /= len(trainLoader)
        writer.add_scalar("Loss/train", trainLoss, epoch)

        ### VALIDATION ###
        validLoss = 0
        for batch_nr, (images, labels) in enumerate(validLoader):

            # Predict
            predictions = network(images)

            # Get loss
            loss = lossFunction(predictions, labels)

            # Save loss for whole epoch
            validLoss += loss.item()

        validLoss /= len(validLoader)
        writer.add_scalar("Loss/valid", validLoss, epoch)

        # Print reuslt of epoch
        print(f'Epoch [{epoch+1}/{epochs}] \t Training Loss: {trainLoss} \t Validation Loss: {validLoss}')

    writer.flush()

In [74]:
def testNetwork(network):
    correct_predictions = 0

    ### TESTING ###
    with torch.no_grad(): 
        for batch_nr, (images, labels) in enumerate(testLoader):
            # Get predictions, convert to number and get the amount of correct predicitons
            predictions = network(images)
            _, predictions = torch.max(predictions, 1) 
            correct_predictions += (predictions == labels).sum().item() 
        
    print(f"Accuracy: {100 * correct_predictions / len(testSet)}%")

In [75]:
learningRate = 0.0001
epochs = 5

In [76]:
networkLeakyRelu = ConvNet(F.leaky_relu)

optimizerSGD = torch.optim.SGD(networkLeakyRelu.parameters(), lr=learningRate)
lossFunction = nn.CrossEntropyLoss()

In [77]:
trainNetwork(networkLeakyRelu, optimizerSGD, lossFunction, epochs, writer)

Epoch [1/5] 	 Training Loss: 2.303406321287155 	 Validation Loss: 2.3035443744659423
Epoch [2/5] 	 Training Loss: 2.300627088814974 	 Validation Loss: 2.300563935756683
Epoch [3/5] 	 Training Loss: 2.29736009529233 	 Validation Loss: 2.2966023067235946
Epoch [4/5] 	 Training Loss: 2.2923789295852184 	 Validation Loss: 2.290067247271538
Epoch [5/5] 	 Training Loss: 2.283422205388546 	 Validation Loss: 2.2776669088602066


In [78]:
testNetwork(networkLeakyRelu)

Accuracy: 12.76%


In [79]:
optimizerAdam = torch.optim.Adam(networkLeakyRelu.parameters(), lr=learningRate)
trainNetwork(networkLeakyRelu, optimizerAdam, lossFunction, epochs, writer)

Epoch [1/5] 	 Training Loss: 1.7643170207962393 	 Validation Loss: 1.6029024189412593
Epoch [2/5] 	 Training Loss: 1.5290641580224038 	 Validation Loss: 1.477316842868924
Epoch [3/5] 	 Training Loss: 1.4442732796836644 	 Validation Loss: 1.4128119117319584
Epoch [4/5] 	 Training Loss: 1.3786085936129093 	 Validation Loss: 1.3682587967962028
Epoch [5/5] 	 Training Loss: 1.3235935923717916 	 Validation Loss: 1.3264432729482651


In [80]:
testNetwork(networkLeakyRelu)

Accuracy: 52.91%


In [81]:
networkTanh = ConvNet(torch.tanh)
trainNetwork(networkTanh, optimizerAdam, lossFunction, epochs, writer)

Epoch [1/5] 	 Training Loss: 2.300424740731716 	 Validation Loss: 2.2999627722501756
Epoch [2/5] 	 Training Loss: 2.300424740612507 	 Validation Loss: 2.2999627767801285
Epoch [3/5] 	 Training Loss: 2.30042474219203 	 Validation Loss: 2.299962767004967
Epoch [4/5] 	 Training Loss: 2.300424739181995 	 Validation Loss: 2.2999627704620362
Epoch [5/5] 	 Training Loss: 2.300424743592739 	 Validation Loss: 2.2999627724885943


In [82]:
testNetwork(networkTanh)

Accuracy: 11.13%
